In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as partial
%matplotlib inline

# 导入paddle库
import paddle
import random

In [13]:
# 导入训练数据和测试数据
df_train = pd.read_csv("train_data_public.csv")
df_test = pd.read_csv("test_public.csv")
df_train.head()

,id,text,BIO_anno,class
0,0,交行14年用过，半年准备提额，却直接被降到1Ｋ，半年期间只T过一次三千，其它全部真实消费，第...,B-BANK I-BANK O O O O O O O O O O B-COMMENTS_N...,0
1,1,单标我有了，最近visa双标返现活动好,B-PRODUCT I-PRODUCT O O O O O O B-PRODUCT I-PR...,1
2,2,建设银行提额很慢的……,B-BANK I-BANK I-BANK I-BANK B-COMMENTS_N I-COM...,0
3,3,我的怎么显示0.25费率，而且不管分多少期都一样费率，可惜只有69k,O O O O O O O O O O B-COMMENTS_N I-COMMENTS_N ...,2
4,4,利率不错，可以撸,B-COMMENTS_N I-COMMENTS_N B-COMMENTS_ADJ I-COM...,1


In [19]:
# 从 http://nlp.stanford.edu/data/glove.6B.zip 下载预训练向量，解压并放入glove文件夹
glove_path = "glove/glove.6B.100d.txt"
embeddings = {}
# 使用utf8编码解码
with open(glove_path, encoding='utf-8') as gf:
    line = gf.readline()
    print("GloVe单行数据：'%s'" % line)

GloVe单行数据：'the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062
'
